In [1]:
import torch
from torch.autograd import Variable
from models.gflstm import GFLSTM
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from data.mag_data_load import mag_load

In [2]:
b_size = 4

trainset = mag_load('./data/T1_class', train = True, download = True)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = b_size, shuffle = True)
testset = mag_load('./data/T1_class', train = False, download = True)
testloader = torch.utils.data.DataLoader(testset, batch_size = 1, shuffle = False)

In [3]:
gf = GFLSTM(850, 100, 2)
lastL = nn.Linear(1000, 44)
criterion = nn.MSELoss()
optimizer = optim.SGD(gf.parameters(),lr = 0.01, momentum = 0.9)

In [5]:
for epoch in range(1):
    for i, data in enumerate(trainloader):
        sig, T = data
        sig, T = Variable(sig), Variable(T)
        optimizer.zero_grad()
        h = [[Variable(torch.zeros(b_size,100)), Variable(torch.zeros(b_size,100))]]
        c = [Variable(torch.zeros(b_size,100)), Variable(torch.zeros(b_size,100))]
        for t in range(11):
            h_next, c = gf(sig, h[t], c)
            h.append(h_next)
        print(h[0][-1])
        out = F.sigmoid(lastL(torch.cat([h[l + 1][-1] for l in range(10)], 2)))
        loss = criterion(out, T)
        loss.backward()
        optimizer.step()

Variable containing:

Columns 0 to 12 
    0     0     0     0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0     0     0     0     0

Columns 13 to 25 
    0     0     0     0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0     0     0     0     0

Columns 26 to 38 
    0     0     0     0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0     0     0

RuntimeError: invalid argument 2: dimension 2 out of range of 2D tensor at /opt/conda/conda-bld/pytorch_1503965122592/work/torch/lib/TH/generic/THTensor.c:24

In [5]:
a = Variable(torch.zeros(2,3,5))
b = Variable(torch.zeros(2,3,1))

a*b

Variable containing:
(0 ,.,.) = 
  0  0  0  0  0
  0  0  0  0  0
  0  0  0  0  0

(1 ,.,.) = 
  0  0  0  0  0
  0  0  0  0  0
  0  0  0  0  0
[torch.FloatTensor of size 2x3x5]